In [ ]:
! pip install geopandas
! pip install pandas_profiling
! pip install ipywidgets
! pip install widgetsnbextension

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import textwrap
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [ ]:
covid19_df = pd.read_csv("../input/covid19-indonesia/covid_19_indonesia_time_series_all.csv")
covid19_df.head(5)

In [ ]:
covid19_df.info()
covid19_df[0:10]

copy dataset

In [ ]:
covid_new = covid19_df.copy()
covid_new.head(5)


melakukan drop kolom yang tidak berguna

In [ ]:
covid_new.drop(["Location ISO Code", "New Cases", "New Deaths", "New Recovered", "New Active Cases", "Province", "Country", "Continent", "Island", "Time Zone", 
                "Special Status", "Total Regencies", "Total Cities", "Total Districts", "Total Urban Villages", "Total Rural Villages", "Area (km2)", 
                "New Cases per Million", "Total Cases per Million","New Deaths per Million", "Total Deaths per Million", "Case Fatality Rate",
                "Case Recovered Rate", "Growth Factor of New Cases","Growth Factor of New Deaths", "City or Regency"],axis=1,inplace=True)

In [ ]:
covid_new.info()

drop data yang double pada kolom Location untuk mendapat update terakhir

In [ ]:
covid_final = covid_new.drop_duplicates(subset='Location', keep="last")
covid_final

In [ ]:
prov = gpd.read_file('../input/dataset-peta-indonesia/prov.shp')
prov.drop(["NAME_0"],axis=1,inplace=True)
prov.rename(columns = {"NAME_1":"Provinsi"}, inplace = True)
prov['Provinsi'].replace({"Jakarta Raya":"DKI Jakarta"},inplace=True)

mengganti nama kolom Location menjadi Provinsi

In [ ]:
covid_final.rename(columns={
    'Location':'Provinsi'
},inplace=True)
covid_final

penggabungan 2 dataset

In [ ]:
covid_peta = prov.merge(covid_final.sort_values(by='Provinsi'),on='Provinsi')


Pembuatan peta

In [ ]:
fig,ax = plt.subplots(4,figsize=(36,36))
plt.style.use('seaborn')
covid_peta.plot(column='Total Cases',cmap='Reds',figsize=(9,36),legend=True,ax=ax[0])
ax[0].set_title('Total kasus',fontsize=15)
covid_peta.plot(column='Total Deaths',cmap='Purples',figsize=(9,36),legend=True,ax=ax[1])
ax[1].set_title('Total Kematian',fontsize=15)
covid_peta.plot(column='Total Recovered',cmap='Greens',figsize=(9,36),legend=True,ax=ax[2])
ax[2].set_title('Total Sembuh',fontsize=15)
covid_peta.plot(column='Total Active Cases',cmap='Blues',figsize=(9,36),legend=True,ax=ax[3])
ax[3].set_title('Total Kasus Aktif',fontsize=15)

#add background color
ax[0].set_facecolor('#e0e0e0')
ax[1].set_facecolor('#e0e0e0')
ax[2].set_facecolor('#e0e0e0')
ax[3].set_facecolor('#e0e0e0')

for i in np.arange(4):
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    for item in ax[i].get_xticklabels():
        item.set_visible(False)
    for item in ax[i].get_yticklabels():
        item.set_visible(False)
    ax[i].set_xticks([])
    ax[i].set_yticks([])
fig.suptitle('Peta Covid-19 Indonesia', fontsize=18)

Top 10 Provinsi dengan Kasus Terbanyak

In [ ]:
total_cases = covid_peta.sort_values(by=['Total Cases'], ascending=False)
total_cases = total_cases[0:10]
plt.figure(figsize=(8,15),dpi=200)
ax = sns.barplot(total_cases['Total Cases'], total_cases['Provinsi'],palette='Paired')
plt.xlabel("Total Cases",fontsize = 15)
plt.ylabel("Provinsi",fontsize = 15)
plt.title('Top 10 provinsi dengan kasus terbanyak',fontsize = 25)
ax.set_yticklabels(textwrap.fill(y.get_text()) for y in ax.get_yticklabels())
ax.tick_params(axis="y", labelsize=15)
for i in ax.patches:
    ax.text(i.get_width() + .05, i.get_y() + 0.5, str(i.get_width()), fontsize = 15, color = 'black')
plt.show()

Top 10 Provinsi dengan kematian karna covid-19 Terbanyak

In [ ]:
total_death = covid_peta.sort_values(by=['Total Deaths'], ascending=False)
total_death = total_death[0:10]
plt.figure(figsize=(8,15),dpi=200)
ax = sns.barplot(total_death['Total Deaths'], total_death['Provinsi'],palette='Paired')
plt.xlabel("Total Deaths",fontsize = 15)
plt.ylabel("Provinsi",fontsize = 15)
plt.title('Top 10 provinsi dengan Kematian karna Covid-19 terbanyak',fontsize = 25)
ax.set_yticklabels(textwrap.fill(y.get_text()) for y in ax.get_yticklabels())
ax.tick_params(axis="y", labelsize=15)
for i in ax.patches:
    ax.text(i.get_width() + .05, i.get_y() + 0.5, str(i.get_width()), fontsize = 15, color = 'black')
plt.show()

Top 10 Provinsi dengan kesembuhan dari covid-19 Terbanyak

In [ ]:
total_recovery = covid_peta.sort_values(by=['Total Recovered'], ascending=False)
total_recovery = total_recovery[0:10]
plt.figure(figsize=(8,15),dpi=200)
ax = sns.barplot(total_recovery['Total Recovered'], total_recovery['Provinsi'],palette='Paired')
plt.xlabel("Total Recovered",fontsize = 15)
plt.ylabel("Provinsi",fontsize = 15)
plt.title('Top 10 provinsi dengan kesembuhan dari  Covid-19 terbanyak',fontsize = 25)
ax.set_yticklabels(textwrap.fill(y.get_text()) for y in ax.get_yticklabels())
ax.tick_params(axis="y", labelsize=15)
for i in ax.patches:
    ax.text(i.get_width() + .05, i.get_y() + 0.5, str(i.get_width()), fontsize = 15, color = 'black')
plt.show()

Top 10 Provinsi dengan kasus aktif covid-19 Terbanyak

In [ ]:
total_active_cases = covid_peta.sort_values(by=['Total Active Cases'], ascending=False)
total_active_cases = total_active_cases[0:10]
plt.figure(figsize=(8,15),dpi=200)
ax = sns.barplot(total_active_cases['Total Active Cases'], total_active_cases['Provinsi'],palette='Paired')
plt.xlabel("Total Active Cases",fontsize = 15)
plt.ylabel("Provinsi",fontsize = 15)
plt.title('Top 10 provinsi dengan kasus aktif Covid-19 terbanyak',fontsize = 25)
ax.set_yticklabels(textwrap.fill(y.get_text()) for y in ax.get_yticklabels())
ax.tick_params(axis="y", labelsize=15)
for i in ax.patches:
    ax.text(i.get_width() + .05, i.get_y() + 0.5, str(i.get_width()), fontsize = 15, color = 'black')
plt.show()